# Parsing WebSIS Student Lists

     Fall TERM 2019-2020      Registration Class List      6.666  U       25-Nov-19

       6.666       Something or Other                      Somebody DeProfessor
                   Standard A - F Grading                  32-G000   555-1234

     MIT ID    Student Name        Course  Y St  Un Grade  Enrolled Sec Email Address
     ------    ------------------  ------  - --- -- -----  -------- --- -------------
     920073587 Martin, Odette Q.      6 3  3 Reg 12          6.666      amet@co.uk
     911887284 Clarke, Cyrus G.       6 3  2 Reg 12          6.666      orci@dignissim.ca

What we want is to parse it to Python data structures. For the first two lines above we want:

In [ ]:
[{'MIT ID': '920073587',
  'Student Name': 'Martin, Odette Q.',
  'Course': '6 3',
  'Y': '3',
  'St': 'Reg',
  'Un': '12',
  'Grade': '',
  'Enrolled': '6.666',
  'Sec': '',
  'Email Address': 'amet@co.uk'},
 {'MIT ID': '911887284',
  'Student Name': 'Clarke, Cyrus G.',
  'Course': '6 3',
  'Y': '2',
  'St': 'Reg',
  'Un': '12',
  'Grade': '',
  'Enrolled': '6.666',
  'Sec': '',
  'Email Address': 'orci@dignissim.ca'} ]

How do we go about doing this?

In [ ]:
def parseWebsis(filename):
    """Read a single WebSIS student-enrollment record from the named file."""
    raise NotImplementedError

First, let's build a simple generator to yield all characters in a text file that we've opened. The argument to `f.read()` is the number of bytes that are returned.

In [ ]:
def characters(f):
    """For f an open file, yield all its characters in order."""
    
    while True:
        c = f.read(1)
        if not c:
            return
        yield c

Here's the handy helper function that is useful to find the dashes and spaces line.

In [ ]:
def allCharactersIn(checkForThese, inThis):
    """Check if all characters of one string are also in the other string."""
    
    for ch in inThis:
        if not ch in checkForThese:
            return False
    return True

The next key ingredient will be processing the line that has a whole bunch of dashes on it, indicating the boundaries of the different fields in the rows that follow.

In [ ]:
def dashSpans(s):
    """Given a string of dashes and nondash characters, return the beginning of each sequence of dashes.
    The output is a list of the numeric positions of those first characters, within the string."""
    
    alreadyInDash = False
    output = []


    return output

In the WebSIS format, when a field value is shorter than the total length allocated to the field, extra spaces appear.  Let's write a function to remove the extras.

In [ ]:
def removeTrailing(s):
    """Return a string that omits trailing spaces and newlines."""
    for i in reversed(range(len(s))):
        # The following check succeeds on the _last_ nonspace character in the string.
        if s[i] != ' ' and s[i] != '\n':
            return s[:i+1]
    return ""

Now we can process the column headers.

In [ ]:
def columnHeaders(s, spans):
    """Given the dash spans of a string, extract the column headers from a
    string standing for the immediately previous line in the file."""
    
    lastSpan = None
    output = []

    for span in spans:
        if lastSpan:
            output.append(removeTrailing(s[lastSpan:span]))
        lastSpan = span

    output.append(removeTrailing(s[lastSpan:]))
    return output

Next, here's how we can process one row of data.

In [ ]:
def removeLeading(s):
    """Return a string that omits leading spaces and newlines."""
    for i in range(len(s)):
        if s[i] != ' ' and s[i] != '\n':
            return s[i:]
    return ""

def trim(s):
    """Remove both leading and trailing spaces and newlines."""
    return removeLeading(removeTrailing(s))

def readRow(spans, headers, s):
    """Given information on field spans and headers, process a single line of text
    into a dictionary with one field per header."""
    output = {}

    for i in range(len(spans)):
        if i < len(spans)-1:
            value = s[spans[i]:spans[i+1]]
        else:
            value = s[spans[i]:]
        output[headers[i]] = trim(value)

    return output

Now we have finished implementing the subroutines that are in `parseWebsis()` and can run it!

In [ ]:
websis = parseWebsis('websis.txt')
websis

# Parsing XML Grades Data

In [ ]:
<grades>
  <psets>
    <pset>
      <number>1</number>
      <student>
        <email>amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk</email>
        <grade>96</grade>
      </student>
      <student>
        <email>adipiscing@lobortisnisi.com</email>
        <grade>91</grade>
      </student>
    </pset>
    <pset>
      <number>2</number>
      <student>
        <email>amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk</email>
        <grade>73</grade>
      </student>
      <student>
        <email>adipiscing@lobortisnisi.com</email>
        <grade>98</grade>
      </student>
    </pset>
  </psets>
  <quizzes>
    <quiz>
      <number>1</number>
      <student>
        <email>amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk</email>
        <grade>92</grade>
      </student>
      <student>
        <email>adipiscing@lobortisnisi.com</email>
        <grade>93</grade>
      </student>
    </quiz>
    <quiz>
      <number>2</number>
      <student>
        <email>amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk</email>
        <grade>22</grade>
      </student>
      <student>
        <email>adipiscing@lobortisnisi.com</email>
        <grade>51</grade>
      </student>
    </quiz>
  </quizzes>
</grades>

What we eventually want in terms of Python data structures for the above XML is this:

In [ ]:
{'psets': {1: {'amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk': 96,
   'adipiscing@lobortisnisi.com': 91},
  2: {'amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk': 73,
   'adipiscing@lobortisnisi.com': 98}},
 'quizzes': {1: {'amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk': 92,
   'adipiscing@lobortisnisi.com': 93},
  2: {'amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk': 22,
   'adipiscing@lobortisnisi.com': 51}}}                                       

We use some ad-hoc code for general XML parsing.  (Actually, we only handle a simple subset of XML, sufficient for this example.)  The algorithm is moderately straightforward, depending at each point on a *tag stack*.  XML documents represent trees, and, at any point in parsing, we are processing some *tag*, or tree node.  The stack at that point stores all the ancestors of the current tag, in order.  As we finish parsing a node, we add that node to the children list of its parent, and we need the stack to find that parent.

In [ ]:
def parseXml(filename):
    """Parse one XML tag (with children) from a file."""
    with open(filename, "r") as f:
        # We maintain a stack of tags that we are still working on.
        # Each is an object that contains a list of children.
        # We start with a dummy tag standing for the document root.
        stack = [{"tag": "root", "children": [], "text": ""}]

        # Now we loop until there is no more content to read.
        while True:
            #Code Stuff here


        # Now we should find the main tag as the sole child of the root node.
        return stack[0]["children"][0]

In [ ]:
grades = parseXml("grades_excerpt.xml")
grades

We postprocess the XML to generate more natural Python data structures, for instance using dictionaries in place of lists of child tags.

In [ ]:
def assignmentFromXml(xml):
    """Convert one assignment (pset or quiz) to a nicer format."""
    
    number = [int(child["text"])
              for child in xml["children"]
              if child["tag"] == "number"][0]

    students = {email["text"]: int(grade["text"])
                for child in xml["children"]
                if child["tag"] == "student"
                for email in child["children"]
                if email["tag"] == "email"
                for grade in child["children"]
                if grade["tag"] == "grade"}

    return {"number": number, "students": students}

def categoryFromXml(xml, singular):
    """Convert one category (psets or quizzes) to a nicer format."""
    
    return {item["number"]: item["students"]
            for child in xml["children"]
            if child["tag"] == singular
            for item in [assignmentFromXml(child)]}

def gradesFromXml(xml):
    """Convert a whole grades database from XML to a nicer format."""
    
    psets = [child
             for child in xml["children"]
             if child["tag"] == "psets"][0]
    quizzes = [child
               for child in xml["children"]
               if child["tag"] == "quizzes"][0]

    return {"psets": categoryFromXml(psets, "pset"),
            "quizzes": categoryFromXml(quizzes, "quiz")}

Bringing it all together:

In [ ]:
def parseGrades(filename):
    """Return the nice version of the grades database found in the file."""
    
    return gradesFromXml(parseXml(filename))

In [ ]:
grades = parseGrades("grades_excerpt.xml")
grades

After all that fuss implementing an XML parser, we could have just used one that comes with Python.  Here's an example showing how.

In [ ]:
import xml.etree.ElementTree as ET

def assignmentFromXmlEasier(xml):
    number = int(xml.find("number").text)
    students = {child.find("email").text: int(child.find("grade").text)
                for child in xml.findall("student")}

    return {"number": number, "students": students}

def categoryFromXmlEasier(xml, singular):
        return {item["number"]: item["students"]
                for child in xml.findall(singular)
                for item in [assignmentFromXmlEasier(child)]}

def gradesFromXmlEasier(xml):
        return {"psets": categoryFromXmlEasier(xml.find("psets"), "pset"),
                "quizzes": categoryFromXmlEasier(xml.find("quizzes"), "quiz")}

def parseGradesEasier(filename):
        return gradesFromXmlEasier(ET.parse(filename).getroot())

In [ ]:
parseGradesEasier('grades.xml')

# Outputting Online Grades Data

In [ ]:
def splitName(name):
    # These variables will store positions in the string.
    comma = None
    firstSpace = None
    secondSpace = None

    for i in range(len(name)):
        if name[i] == ',' and not comma:
            comma = i
        elif name[i] == ' ' and not firstSpace:
            firstSpace = i
        elif name[i] == ' ':
            secondSpace = i

    return name[:comma], name[firstSpace+1:secondSpace], name[secondSpace+1:secondSpace+2]

In [ ]:
splitName('Doe, John Q.')

It turns out that there's a much simpler way to do this kind of string matching, using Python's standard library and an idea called _regular expressions_.

In [ ]:
import re

def splitNameWithRegex(name):
    m = re.search("(.*), (.*) (.*)\\.", name)
    return m.group(1), m.group(2), m.group(3)

In [ ]:
splitNameWithRegex('Doe, John Q.')

The next important ingredient is the one that actually computes the letter grade! We need to compute percentages and map them to letters. This is not the grading policy we will use this semester!

In [ ]:
def averageToLetter(n):
    if n >= 90:
        return "A"
    elif n >= 80:
        return "B"
    elif n >= 70:
        return "C"
    elif n >= 60:
        return "D"
    else:
        return "A+"

To keep this simple, we'll weight all psets/quizzes equally.

In [ ]:
def studentAverage(email, grades):
    total = 0

    for pset in grades["psets"].values():
        if email in pset:
            total += pset[email]

    for quiz in grades["quizzes"].values():
        if email in quiz:
            total += quiz[email]

    return total / (len(grades["psets"]) + len(grades["quizzes"]))

def studentGrade(email, grades):
    return averageToLetter(studentAverage(email, grades))

In [ ]:
studentAverage('aliquet.vel@Vestibulum.org', grades)

In [ ]:
studentGrade('aliquet.vel@Vestibulum.org', grades)

Now the end of our long journey: outputting the file suitable for OGS import.

In [ ]:
def outputGrades(filename, students, grades):
    with open(filename, "w") as f:
        f.write("Last Name,First Name,Middle,MIT ID,Subject #,Section #,Grade,Units,Comment\n")

        for student in students:
            last, first, middle = splitName(student["Student Name"])
            f.write(last + "," + first + "," + middle + "," \
                    + student["MIT ID"] + "," \
                    + student["Enrolled"] + "," \
                    + student["Sec"] + "," \
                    + studentGrade(student["Email Address"], grades) + "," \
                    + student["Un"] + "," \
                    + "\n")

In [ ]:
outputGrades('ogs.csv', websis, grades)